- [ ] why is the count value when basic sorting is done different from gsheets wala value 
- [ ] TODO-Add error testing to make sure the columns are in the exact same order
- [ ] The above row is from the first method of trying there is a variance in the number of rows a huge variance
- [ ] TODO-Based on Freq eff date and disc date make a counter as unit tests to check data is correct
- [ ] TODO-the progrmamming needs to be converted from a procedural to functional programmming paradigm
-[ ] TODO - performace hit on adding columns invidually in sliced dataframe versus to total dataframe
- [ ] text comment a134phase 6e2015 01 feb is a edge case for single flight per day per rotation 
- [ ] TODO : each set of textcomment and day needs to tested for testcomment set data is compliant 
- [ ]   :interrobang: 🤒
- #Improvement Action:
- #Resultant Time:"
- df2._is_view
- df._is_copy


In [2]:
%%time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import datetime
import pickle
import re
scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name("crudg.json", scope)
client = gspread.authorize(creds)
sheet = client.open("flightsched").get_worksheet(2)
dataframe = pd.DataFrame(sheet.get_all_records())
# Improvement Action :
# Resultant Time:


Wall time: 5.88 s


In [2]:
# Set ipython's max row display
pd.set_option('display.max_row', 50)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

In [16]:
dataframe.head()
# bad idea to use loc for scalar access 
startDate = dataframe.loc[0,:][9]
endDate = dataframe.loc[0,:][10]
testDate =pd.date_range(start=startDate, end= endDate ,freq="D").tolist()
int(dataframe.loc[0,:][4].split(":")[0])

7

In [11]:
%%time
def formatTidyData(paramDataframe):
    '''
    input: dataframe 
    
    transform: take each row of input dataframe / make each row of output for each valid date between eff and             discontinue date 

    output dataframe 
    '''
    Row_list =paramDataframe.values.tolist()
    masterList =[]
    for xray in Row_list:
            startDate = xray[9]+"-"+xray[4]
            startDate = pd.to_datetime(startDate,format='%d-%m-%y-%h:%M', infer_datetime_format=True, utc=False)
            endDate = xray[10]+"-"+xray[4]
            endDate = pd.to_datetime(endDate,format='%d-%m-%y-%h:%M', infer_datetime_format=True, utc=False)
            #TODO : refactor startDate and Date can be directly given as strings 
            requiredRange =pd.date_range(startDate, endDate, freq='D').tolist()
            daysList = str(xray[2]).replace(".","")
            
            for alpha in requiredRange:
                # weekday starts at 0 hence 1 is added 
                if str(alpha.weekday()+1) in daysList:
                    # TODO : refactor use .replace method to set required minutes and hours 
                    depDate = str(alpha.date())+"-"+xray[4]
                    depDate = pd.to_datetime(depDate,format='%d-%m-%y-%h:%M', infer_datetime_format=True, utc=False)
                    arrDate = str(alpha.date())+"-"+xray[6]
                    arrDate = pd.to_datetime(arrDate,format='%d-%m-%y-%h:%M', infer_datetime_format=True, utc=False)
                    masterList.append([xray[0],xray[1],xray[3],xray[5],xray[7],xray[8],xray[11],depDate,arrDate])
    return pd.DataFrame(masterList,columns=["Text_Comment","Flt_Desg","Dept_Arp","Arvl_Arp","Subfleet","Block_Time","Service_Type","aocs_dep","aocs_arr"])

        

#Wall time: 34.8 s
# 44.8 s ± 4.3 s per loop (mean ± std. dev. of 7 runs, 1 loop each) with time it
# 41.6 s ± 6.39 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

Wall time: 0 ns


In [12]:
%%time
testpd = formatTidyData(dataframe)


Wall time: 45.7 s


In [13]:
testpd.head()

,Text_Comment,Flt_Desg,Dept_Arp,Arvl_Arp,Subfleet,Block_Time,Service_Type,aocs_dep,aocs_arr
0,A001,6E 6269,BLR,MAA,6E 32S,1:05,J,2021-03-28 07:10:00,2021-03-28 08:15:00
1,A001,6E 6269,BLR,MAA,6E 32S,1:05,J,2021-03-29 07:10:00,2021-03-29 08:15:00
2,A001,6E 6269,BLR,MAA,6E 32S,1:05,J,2021-03-30 07:10:00,2021-03-30 08:15:00
3,A001,6E 6269,BLR,MAA,6E 32S,1:05,J,2021-03-31 07:10:00,2021-03-31 08:15:00
4,A001,6E 6269,BLR,MAA,6E 32S,1:05,J,2021-04-01 07:10:00,2021-04-01 08:15:00


In [14]:
%%time
# list(array of all days in our data )
uniqueDates = np.sort(testpd["aocs_dep"].dt.date.unique())
#list of all unique airports 
aiportList = np.sort(testpd["Dept_Arp"].unique())

# list of Comment List which are unique 
textCommentList = testpd["Text_Comment"].unique()
#list of all unique flightNumbers
flightNumberUniqueList = np.sort(testpd["Flt_Desg"].unique())

# adding column "engg_arr" and "deptype"
testpd["engg_arr"]= np.nan
testpd["engg_dep"]= np.nan
testpd["deptype"]= np.nan


# making block time as time delta in python 
testpd["Block_Time"]= testpd["Block_Time"].astype("str")
splitString = testpd["Block_Time"].str.split(':', expand=True).astype(int)
testpd["Block_Time"] = pd.to_timedelta(splitString[0], unit='h') + pd.to_timedelta(splitString[1], unit='m')


# Adding datatypes of columns for performance improbement 
testpd= testpd.astype({"deptype":"string","engg_arr":"datetime64[ns]","engg_dep":"datetime64[ns]","Flt_Desg":"string","Block_Time":"timedelta64[ns]"})

# reorder columns
testpd= testpd[['Dept_Arp', 'engg_arr', 'Flt_Desg', 'Text_Comment', 'Subfleet','Arvl_Arp',  'aocs_dep','engg_dep', 'aocs_arr','deptype',  'Block_Time','Service_Type']]

# testpd.info()


Wall time: 1.13 s


In [15]:
# make last row 

# def make_last_row(onedf):
#     deepcopy = onedf.copy()
#     temp_false_list = []
#     if len(deepcopy) == 1:
#         return deepcopy
#     else:
#         for india in range(onedf.shape[1]):
#             temp_false_list.append(np.nan)
#         deepcopy.loc[len(onedf)] = temp_false_list
#         # tempSeries = pd.DataFrame(temp_false_list, columns)

#         # deepcopy = pd.concat([deepcopy,tempSeries], ignore_index=True)
#         deepcopy.reset_index(drop=True, inplace=True)
#         deepcopy.at[(len(deepcopy)-1), "Dept_Arp" ] =deepcopy.at[(len(deepcopy)-2), "Arvl_Arp" ]
#         deepcopy.at[(len(deepcopy)-1), "Flt_Desg" ] =deepcopy.at[(len(deepcopy)-2), "Flt_Desg" ]
#         deepcopy.at[(len(deepcopy)-1), "engg_arr" ] =deepcopy.at[(len(deepcopy)-2), "aocs_dep"]
#         deepcopy.at[(len(deepcopy)-1), "Text_Comment" ] =deepcopy.at[(len(onedf)-2), "Text_Comment"]
#         deepcopy.at[(len(deepcopy)-1), "Subfleet" ] =deepcopy.at[(len(onedf)-2), "Subfleet"]
        
#         return deepcopy






In [16]:

# def AddBaseArrDep(paramdf):
#     '''
#     Modify first and last row for base Departure and base arrival model 
#     '''
#     onedf = paramdf.copy()
#     onedf.reset_index(drop=True, inplace=True)
#     if len(onedf)==1:
#         # edge case of only one departure for one text comment
#         onedf.at[0,"deptype"] = "baseDep"
#         onedf.at[0,"engg_dep"] = onedf.at[0,"aocs_dep"]
#         return onedf
#     else:

#         for xray in range(0,len(onedf)):
#             if xray  == 0:
#                 onedf.at[xray,"deptype"] = "baseDep"
#                 onedf.at[xray,"engg_dep"] = onedf.at[xray,"aocs_dep"]
#             elif xray == len(onedf)-1:
#                 onedf.at[xray,"deptype"] = "baseArr"
#             else:
#                 onedf.at[xray,"engg_arr"] = onedf.at[(xray-1),"aocs_arr"]
#                 onedf.at[xray,"engg_dep"] = onedf.at[(xray),"aocs_dep"]
#                 onedf.at[xray,"deptype"] = "Transit"
#         return onedf





In [17]:
 
 def addlastRowArrDep(paramdf):
     # temporary make columns consistent 
     paramdf = paramdf[['Dept_Arp', 'engg_arr', 'Flt_Desg', 'Text_Comment', 'Subfleet','Arvl_Arp',  'aocs_dep','engg_dep', 'aocs_arr','deptype',  'Block_Time','Service_Type']]
     # edge case of only one departure for one text comment
     if  (len(paramdf.to_numpy()) == 1):
         temparray= paramdf.to_numpy()
         temparray[0][9]= "baseDep"
         temparray[0][7] = temparray[0][6]
         return temparray
     else:
            temparray = np.append(
                paramdf.to_numpy(),np.full((1, 12), np.nan),axis=0)
            temparray[len(temparray)-1][0:2]=temparray[len(temparray)-2][5:7]
            temparray[len(temparray)-1][2:5]=temparray[len(temparray)-2][2:5]
            for xray in range(0,len(temparray)):
                if xray == 0:
                    # onedf.at[xray,"deptype"] = "baseDep"
                    temparray[xray][9]= "baseDep"
                    # onedf.at[xray,"engg_dep"] = onedf.at[xray,"aocs_dep"]
                    temparray[xray][7] = temparray[xray][6]
                elif  xray == len(temparray)-1:
                    # onedf.at[xray,"deptype"] = "baseArr"
                    temparray[xray][9]= "baseArr"
                else:
                    # onedf.at[xray,"engg_arr"] = onedf.at[(xray-1),"aocs_arr"]
                    temparray[xray][1] = temparray[xray-1][8]
                    # onedf.at[xray,"engg_dep"] = onedf.at[(xray),"aocs_dep"]
                    temparray[xray][7] = temparray[xray][6]
                    # onedf.at[xray,"deptype"] = "Transit"
                    temparray[xray][9]= "Transit"
            return temparray    








In [18]:
# %%time
# y_df =testpd.loc[(testpd["Text_Comment"]== textCommentList[0] ) & (testpd["aocs_dep"].dt.date==uniqueDates[3])]
# y_df= y_df.copy()
# y_df =  make_last_row(y_df)
# y_df = AddBaseArrDep(y_df)
# y_df = y_df[['Dept_Arp', 'engg_arr', 'Flt_Desg', 'Text_Comment', 'Subfleet','Arvl_Arp',  'aocs_dep','engg_dep', 'aocs_arr','deptype',  'Block_Time','Service_Type']]
# y_df

In [19]:
# %%time
# zuludf =  testpd.loc[(testpd["Text_Comment"]== textCommentList[0] ) & (testpd["aocs_dep"].dt.date==uniqueDates[3])]
# zuludf=zuludf.copy()
# temparray = addlastRowArrDep(zuludf)
# masterArray =[]           
# [masterArray.append(zulu)  for zulu in temparray ]
# pd.DataFrame(masterArray, columns = ['Dept_Arp', 'engg_arr', 'Flt_Desg', 'Text_Comment', 'Subfleet','Arvl_Arp',  'aocs_dep','engg_dep', 'aocs_arr','deptype',  'Block_Time','Service_Type'])




In [20]:
%%time
masterList=[]
for xray in textCommentList:
    for alpha in uniqueDates:
        # filtering dataframe on CommentValue and uniqueDateValue
        x_df =testpd.loc[(testpd["Text_Comment"]== xray) & (testpd["aocs_dep"].dt.date==alpha)]
        # check if dataframe is empty
        if len(x_df) == 0:
            continue
        # sorting by aocs_dep for safety 
        x_df= x_df.sort_values(by=["aocs_dep"], ascending=True)
        # # calling function "make_last_row"
        # x_df = make_last_row(x_df)
        # # calling function
        # x_df  = AddBaseArrDep(x_df)
        # # to do -: 
        # for zulu in x_df.values.tolist():
        #     masterList.append(zulu)
        # for performance shifting to np array 
        x_df = addlastRowArrDep(x_df)
        [masterList.append(zulu)  for zulu in x_df ] 






#Wall time: 6min 25s with proper categories defined 
# Wall time: 6min 25s string
# Wall time: 5min 25s accesor changed to at 
# Wall time: 4min 55s 3 categories given as string 
# Wall time: 6min 16s inplace = True
# <timed exec>:10: SettingWithCopyWarning: 
# A value is trying to be set on a copy of a slice from a DataFrame

# See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
# Wall time: 3min 44s
# Wall time: 4min 1s

#Wall time: 3min 44s inplace = True removed instaed of to values two numpy used
#Wall time: 5min 13s old method 
# Wall time: 5min 35s month April

Wall time: 5min 35s


In [21]:
%%time
cleanDf = pd.DataFrame(masterList,columns=['Dept_Arp', 'engg_arr', 'Flt_Desg', 'Text_Comment', 'Subfleet','Arvl_Arp',  'aocs_dep','engg_dep', 'aocs_arr','deptype',  'Block_Time','Service_Type'])
cleanDf
# current time using nparray 50 ms using array 
# current time using pythonlist  = Wall time: 175 ms
# Wall time: 412 ms with first performance improvement
# Wall time: 525 ms with all non time as strngs
# Wall time: 343 ms accesor changed to at 
# Wall time: 543 ms with some as string
# Wall time: 493 ms
# 40045 rows × 12 columns using the numpy method 
# Wall time: 274 ms old method 
# 40044 rows × 12 columns old numpy method

Wall time: 660 ms


,Dept_Arp,engg_arr,Flt_Desg,Text_Comment,Subfleet,Arvl_Arp,aocs_dep,engg_dep,aocs_arr,deptype,Block_Time,Service_Type
0,BLR,NaT,6E 6269,A001,6E 32S,MAA,2021-03-28 07:10:00,2021-03-28 07:10:00,2021-03-28 08:15:00,baseDep,0 days 01:05:00,J
1,MAA,2021-03-28 08:15:00,6E 0991,A001,6E 32S,DEL,2021-03-28 08:50:00,2021-03-28 08:50:00,2021-03-28 11:50:00,Transit,0 days 03:00:00,J
2,DEL,2021-03-28 11:50:00,6E 0657,A001,6E 32S,PAT,2021-03-28 12:40:00,2021-03-28 12:40:00,2021-03-28 14:20:00,Transit,0 days 01:40:00,J
3,PAT,2021-03-28 14:20:00,6E 0286,A001,6E 32S,DEL,2021-03-28 14:50:00,2021-03-28 14:50:00,2021-03-28 16:25:00,Transit,0 days 01:35:00,J
4,DEL,2021-03-28 16:25:00,6E 0286,A001,6E 32S,PNQ,2021-03-28 17:20:00,2021-03-28 17:20:00,2021-03-28 19:15:00,Transit,0 days 01:55:00,J
...,...,...,...,...,...,...,...,...,...,...,...,...
47582,BLR,2021-04-30 15:20:00,6E 7162,ATR_001,6E ATR,HBX,2021-04-30 15:55:00,2021-04-30 15:55:00,2021-04-30 17:25:00,Transit,0 days 01:30:00,J
47583,HBX,2021-04-30 17:25:00,6E 7995,ATR_001,6E ATR,GOI,2021-04-30 17:55:00,2021-04-30 17:55:00,2021-04-30 18:40:00,Transit,0 days 00:45:00,J
47584,GOI,2021-04-30 18:40:00,6E 7996,ATR_001,6E ATR,HBX,2021-04-30 19:10:00,2021-04-30 19:10:00,2021-04-30 20:00:00,Transit,0 days 00:50:00,J
47585,HBX,2021-04-30 20:00:00,6E 7161,ATR_001,6E ATR,BLR,2021-04-30 20:30:00,2021-04-30 20:30:00,2021-04-30 21:45:00,Transit,0 days 01:15:00,J


In [15]:
# %%timeit
cleanDf.dtypes

Dept_Arp                 object
engg_arr         datetime64[ns]
Flt_Desg                 object
Text_Comment             object
Subfleet                 object
Arvl_Arp                 object
aocs_dep         datetime64[ns]
engg_dep         datetime64[ns]
aocs_arr         datetime64[ns]
deptype                  object
Block_Time      timedelta64[ns]
Service_Type             object
dtype: object

In [16]:
# Adding datatypes of columns for performance improbement 
# testpd= testpd.astype({"deptype":"category","Service_Type":"category","Dept_Arp":"category","Arvl_Arp":"category","Text_Comment":"category","Subfleet":"category","engg_arr":"datetime64[ns]","engg_dep":"datetime64[ns]","Flt_Desg":"category","Block_Time":"timedelta64[ns]"})

- [ ] A juypter notebook needs to be tested for two things memory and performance 

In [22]:
# pickle_List = [dataframe,days_sorted_list,dep_airport_list]
pickle.dump(cleanDf, open("apr_21.p", "wb"))

gamma = pickle.load(open("apr_21.p", "rb"))

gamma.equals(cleanDf)

True

In [19]:
alpha = pickle.load(open("gsheet_pickle.p", "rb"))


In [20]:
# testpd.info()
# onedf=testpd.loc[(testpd["Text_Comment"]=="ATR_55")]
# onedf.info()
fourpd = testpd["Text_Comment"].value_counts().reset_index().sort_values('index')
fourpd = fourpd.reset_index(drop=True)
fourpd= fourpd.rename(columns={"Text_Comment":"jebahComment"})



In [21]:
greta =alpha[0]

threePd=greta["Text Comment"].value_counts().reset_index().sort_values('index')
threePd["Text Comment"]=threePd["Text Comment"].astype(int).floordiv(2)
threePd = threePd.reset_index(drop=True)
threePd = threePd.rename(columns={"Text Comment":"GretaComment"})
# 


In [22]:
fourpd["GretaComment"]=threePd["GretaComment"]
fourpd["reqdiff"]= fourpd["GretaComment"]- fourpd["jebahComment"]
fourpd=fourpd[fourpd.reqdiff != 0].sort_values("reqdiff").reset_index(drop=True)
fourpd



,index,jebahComment,GretaComment,reqdiff
0,A124,364,252,-112
1,A126,224,112,-112
2,A320_31,112,56,-56
3,A4,168,112,-56
4,ATR021,216,160,-56
...,...,...,...,...
46,A19,72,48,-24
47,A99,104,80,-24
48,A320_831,184,164,-20
49,A103,96,80,-16


- Trying to work with generator 

In [46]:
testDate =pd.date_range(start='1/1/2021', freq="MS",periods=12).strftime("%b-%y").tolist()
testDate

['Jan-21',
 'Feb-21',
 'Mar-21',
 'Apr-21',
 'May-21',
 'Jun-21',
 'Jul-21',
 'Aug-21',
 'Sep-21',
 'Oct-21',
 'Nov-21',
 'Dec-21']

In [45]:
cleanDf.loc[:,"engg_dep"].min()

Timestamp('2021-03-28 00:05:00')